# Final model - preprocessing and classification

In [1]:
import pandas as pd
import re
import string
import unicodedata

from transformers import pipeline
from nltk.tokenize import word_tokenize
from googletrans import Translator

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv("../notebooks/tweets_clean_en.csv")

## Preprocessing

In [2]:
#remove canais de comunicação
organizations = ["TSFRadio","observadorpt", "Lusa_noticias", "dntwit",
                      "tvi24pt","revistaSABADO", "Renascenca", "RTPNoticias",
                  "SolOnline","dinheirovivo","Visao_pt","tvi","rtppt","govpt",
                  "RTP2","RTP1","ionline","SICNoticias", "Publico","JornalNoticias",
                  "expresso", "ojeconimico","tv124ultimas","ojogo","cmjornal","Record_Portugal",
                  "sapo","RTPNoticias", "saude_pt","SNS_Portugal", "DGSaude","ainterna_pt","antoniocostapm"
            ]

df = df[~df['username'].isin(organizations)]

In [3]:
stopwords = {"coronavirus","covid19", "covid19pt","covid","CoronaVirus","FiqueEmCasa", "fiqueemcasa", "Covid19","StayAtHome",
            "Coronavirus.","COVID2019", "COVID19" "Coronavirus", "STOPCOVID19", "ficaemcasa"}

# Function to clean Portuguese text and tokenize it
def preProcessing(text):
    #text = text.lower() # convert text to lower-case
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
    text = word_tokenize(text) # remove repeated characters
    word_list = [word for word in text if word not in stopwords]
    text = ' '.join(word_list)
    return text

translator = Translator()

# Function to translate the current text to english
def Translation(text):
    if  translator.detect(text) != 'en':
        traducao = translator.translate(text, dest='en')
        return traducao.text
    else:
        return text

In [4]:
df.describe()

,id,date,username,text,hashtag,geo
count,21140,21140,21140,21140,4830,1
unique,20852,20792,6524,21140,2985,1
top,1251971097593106432,2020-03-26 14:19:32+00:00,erassoci,Já to esperando junho do ano que vem pra curtir esse nos alive que o corona acabou com os planos dos amigos esse ano,#COVID19,geo
freq,2,3,164,1,299,1


In [5]:
df['text_clean'] = df['text'].apply(preProcessing)

In [6]:
df.head()

,id,date,username,text,hashtag,geo,text_clean
0,1213243065336582144,2020-01-03 23:37:42+00:00,adreciclarte,Olha que o Corona também está a ficar bonzinho. Se o deixarem jogar no seu lugar.,NaN,NaN,Olha que o Corona também está a ficar bonzinho . Se o deixarem jogar no seu lugar .
1,1213599942444539906,2020-01-04 23:15:48+00:00,food_i_do,"Que escândalo! Parece que só agora é que o árbitro não foi ver as imagens. O golo do Porto que elimina o Santa clara é precedido de falta gritante do corona, nao assinalada, e o VAR nada, dolce fare niente. Mas só hoje é que o VAR FALHOU, não foi? Venha daí a BERRARIA.",NaN,NaN,"Que escândalo ! Parece que só agora é que o árbitro não foi ver as imagens . O golo do Porto que elimina o Santa clara é precedido de falta gritante do corona , nao assinalada , e o VAR nada , dolce fare niente . Mas só hoje é que o VAR FALHOU , não foi ? Venha daí a BERRARIA ."
2,1213939014488788992,2020-01-05 21:43:09+00:00,CRbeiro,Corona meu mexicanozinho tu és do caralho pa!! Te quiero,NaN,NaN,Corona meu mexicanozinho tu és do caralho pa ! ! Te quiero
3,1213891469771247621,2020-01-05 18:34:14+00:00,dudzs16,Não temos a jogar um caralho. A substituição do Pepe foi uma desgraça para nós. Odeio o Marcano. O Bruno Fernandes é um porco. Corona parte tantos rins que o querem partir a ele.,NaN,NaN,Não temos a jogar um caralho . A substituição do Pepe foi uma desgraça para nós . Odeio o Marcano . O Bruno Fernandes é um porco . Corona parte tantos rins que o querem partir a ele .
4,1213890252374466562,2020-01-05 18:29:23+00:00,luisgomes__10,"Digo e volto a dizer, o Corona não pode jogar a lateral",NaN,NaN,"Digo e volto a dizer , o Corona não pode jogar a lateral"


In [22]:
tweets = list(df["text_clean"].to_numpy())

translations = translator.translate(tweets, dest='en')

In [25]:
translations_list = []

for translation in translations:
    translations_list.append(translation.text)

In [26]:
print(translations_list)

['Look, Corona is also looking good. If they let you play in your place.', "What a scandal! It seems that it is only now that the referee has not gone to see the images. The Porto goal that eliminates Santa clara is preceded by a glaring corona foul, unmarked, and VAR nothing, dolce fare niente. But only today did VAR FAIL, wasn't it? Come from there to BERRARIA.", 'Corona my little mexican you are fucking awesome! ! I want', "We don't have to play a fuck. Pepe's replacement was a disgrace for us. I hate Marcano. Bruno Fernandes is a pig. Corona breaks so many kidneys that they want to leave him.", 'I say and say again, Corona cannot play the side', 'corona and nakajima are humiliating sporting', 'Corona is just underwear', "`` Go with him Corona ... After all it's Octávio '' onjoanarita", 'And this Corona pass? Fds that beautiful', 'A Corona manual pass for Porto to jump ahead with Marega', 'Bitch from corona pass, beautiful thing', 'What a beautiful pass from Corona', 'That pass of t

In [27]:
df['text_en'] = translations_list

## Classification

In [2]:
# Allocate a pipeline for sentiment-analysis
nlp = pipeline('sentiment-analysis')

def getSentiment(sentence):
  sentiment = nlp(sentence)[0]['label']
  return sentiment

def getScore(sentence):
  score = nlp(sentence)[0]['score']
  return score

df['classification'] = df['text_en'].apply(getSentiment)
df['score'] = df['text_en'].apply(getScore)

All model checkpoint weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [3]:
df.head()

,Unnamed: 0,id,date,username,text,hashtag,geo,text_clean,text_en,classification,score
0,0,1213243065336582144,2020-01-03 23:37:42+00:00,adreciclarte,Olha que o Corona também está a ficar bonzinho. Se o deixarem jogar no seu lugar.,NaN,NaN,Olha que o Corona também está a ficar bonzinho . Se o deixarem jogar no seu lugar .,"Look, Corona is also looking good. If they let you play in your place.",POSITIVE,0.999872
1,1,1213599942444539906,2020-01-04 23:15:48+00:00,food_i_do,"Que escândalo! Parece que só agora é que o árbitro não foi ver as imagens. O golo do Porto que elimina o Santa clara é precedido de falta gritante do corona, nao assinalada, e o VAR nada, dolce fare niente. Mas só hoje é que o VAR FALHOU, não foi? Venha daí a BERRARIA.",NaN,NaN,"Que escândalo ! Parece que só agora é que o árbitro não foi ver as imagens . O golo do Porto que elimina o Santa clara é precedido de falta gritante do corona , nao assinalada , e o VAR nada , dolce fare niente . Mas só hoje é que o VAR FALHOU , não foi ? Venha daí a BERRARIA .","What a scandal! It seems that it is only now that the referee has not gone to see the images. The Porto goal that eliminates Santa clara is preceded by a glaring corona foul, unmarked, and VAR nothing, dolce fare niente. But only today did VAR FAIL, wasn't it? Come from there to BERRARIA.",NEGATIVE,0.996998
2,2,1213939014488788992,2020-01-05 21:43:09+00:00,CRbeiro,Corona meu mexicanozinho tu és do caralho pa!! Te quiero,NaN,NaN,Corona meu mexicanozinho tu és do caralho pa ! ! Te quiero,Corona my little mexican you are fucking awesome! ! I want,POSITIVE,0.999818
3,3,1213891469771247621,2020-01-05 18:34:14+00:00,dudzs16,Não temos a jogar um caralho. A substituição do Pepe foi uma desgraça para nós. Odeio o Marcano. O Bruno Fernandes é um porco. Corona parte tantos rins que o querem partir a ele.,NaN,NaN,Não temos a jogar um caralho . A substituição do Pepe foi uma desgraça para nós . Odeio o Marcano . O Bruno Fernandes é um porco . Corona parte tantos rins que o querem partir a ele .,We don't have to play a fuck. Pepe's replacement was a disgrace for us. I hate Marcano. Bruno Fernandes is a pig. Corona breaks so many kidneys that they want to leave him.,NEGATIVE,0.998038
4,4,1213890252374466562,2020-01-05 18:29:23+00:00,luisgomes__10,"Digo e volto a dizer, o Corona não pode jogar a lateral",NaN,NaN,"Digo e volto a dizer , o Corona não pode jogar a lateral","I say and say again, Corona cannot play the side",NEGATIVE,0.997583


In [4]:
df.to_csv('tweets_en_classification.csv')